In [112]:
from statsbombpy import sb
import pandas as pd
import numpy as np
import networkx as nx

In [113]:
df = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)

C:\Users\thors\AppData\Roaming\Python\Python312\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [114]:
xT = pd.read_csv("https://raw.githubusercontent.com/AKapich/WorldCup_App/main/app/xT_Grid.csv", header=None)
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

In [115]:
def get_xT(df, event_type):
    df = df[df['type']==event_type]
    
    df['x'], df['y'] = zip(*df['location'])
    df['end_x'], df['end_y'] = zip(*df[f'{event_type.lower()}_end_location'])

    df[f'start_x_bin'] = pd.cut(df['x'], bins=xT_cols, labels=False)
    df[f'start_y_bin'] = pd.cut(df['y'], bins=xT_rows, labels=False)
    df[f'end_x_bin'] = pd.cut(df['end_x'], bins=xT_cols, labels=False)
    df[f'end_y_bin'] = pd.cut(df['end_x'], bins=xT_rows, labels=False)
    df['start_zone_value'] = df[[f'start_x_bin', f'start_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
    df['end_zone_value'] = df[[f'end_x_bin', f'end_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
    df['xT'] = df['end_zone_value']-df['start_zone_value']

    return df[['xT', 'minute', 'team','match_id']]

In [116]:
from copy import deepcopy
aux_df  = deepcopy(df)

aux_df = aux_df[aux_df['type']=='Pass']

In [117]:
aux_df['x'], aux_df['y'] = zip(*aux_df['location'])
aux_df['end_x'], aux_df['end_y'] = zip(*aux_df[f'pass_end_location'])

aux_df[f'start_x_bin'] = pd.cut(aux_df['x'], bins=xT_cols, labels=False)
aux_df[f'start_y_bin'] = pd.cut(aux_df['y'], bins=xT_rows, labels=False)
aux_df[f'end_x_bin'] = pd.cut(aux_df['end_x'], bins=xT_cols, labels=False)
aux_df[f'end_y_bin'] = pd.cut(aux_df['end_x'], bins=xT_rows, labels=False)

In [118]:
aux_df['start_zone_value'] = aux_df[[f'start_x_bin', f'start_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
aux_df['end_zone_value'] = aux_df[[f'end_x_bin', f'end_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)

aux_df['xT'] = aux_df['end_zone_value']-aux_df['start_zone_value']

C:\Users\thors\AppData\Local\Temp\ipykernel_8980\1174840208.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux_df['start_zone_value'] = aux_df[[f'start_x_bin', f'start_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
C:\Users\thors\AppData\Local\Temp\ipykernel_8980\1174840208.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux_df['end_zone_value'] = aux_df[[f'end_x_bin', f'end_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)


In [119]:
xT_data = pd.concat([get_xT(df=df, event_type='Pass'), get_xT(df=df, event_type='Carry')], axis=0)
xT_data = xT_data[xT_data['xT'] > 0.001]

C:\Users\thors\AppData\Local\Temp\ipykernel_8980\662369974.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['x'], df['y'] = zip(*df['location'])
C:\Users\thors\AppData\Local\Temp\ipykernel_8980\662369974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'], df['y'] = zip(*df['location'])
C:\Users\thors\AppData\Local\Temp\ipykernel_8980\662369974.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

In [120]:
#Bin minutes into time intervals so minute 3 is now minute 0, minute 14 is minute 10 and so on
xT_data['minute_interval'] = ((xT_data['minute']) // 5) * 5 

#Group by the team, match and minute interval and calculate the sum of xT for each
xT_sums = xT_data.groupby(["team","match_id",'minute_interval'])['xT'].sum().reset_index()

In [121]:
#Filter for the xT values of Bayer Leverkusen
bayer_xt = xT_sums[xT_sums['team'] == 'Bayer Leverkusen']
#Make new column name before merging 
bayer_xt = bayer_xt.rename(columns={'xT': 'bayer_xT'})

df_merged = xT_sums.merge(bayer_xt, on=['match_id', 'minute_interval'], how='left')

#Calculate momentum for each row, also in the rows where Bayer Lverkusen's "xT" and "bayer_xt" is the same (explain later)
df_merged['momentum'] = df_merged['bayer_xT'] - df_merged['xT']

#Remove rows where momentum is 0 as this is where xT of Bayer Leverkusen is subtracted from itself 
#(There is not an instance where Bayer Leverkusen xT and opponent xT is equal so this isnt a problem)
momentum_df = df_merged[df_merged["momentum"]!=0]

In [123]:
#Make ids based off of the game and minute interval
momentum_df["id"] = momentum_df["match_id"].astype(str) + momentum_df["minute_interval"].astype(str)

C:\Users\thors\AppData\Local\Temp\ipykernel_8980\3464649215.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentum_df["id"] = momentum_df["match_id"].astype(str) + momentum_df["minute_interval"].astype(str)


In [124]:
momentum_df = momentum_df[["id","momentum"]]

In [125]:
momentum_df

,id,momentum
0,38951940,0.101633
1,38951945,0.051752
2,389519410,0.086223
3,389519415,0.171258
4,389519420,-0.096416
...,...,...
1305,389526665,0.076905
1306,389526670,0.062513
1307,389526675,0.161248
1308,389526680,0.168021


In [126]:
momentum_df.to_csv("momentum_data.csv", index=False)